# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending a question to it and getting it scored for matches in the original questions.

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [3]:
# Please modify the below as you see fit
resource_group = "maboumlaksd" 
aks_name = "maboumlaksd"
location = "eastus"

docker_login = 'fboylu'
image_name = docker_login + '/mlaksdep' # 'fboylu/mlaksdep' Use this image if you want to skip creating your own container
selected_subscription = "'Boston-DS-Brandon-Dev'" # If you have multiple subscriptions select the subscription you want to use 

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [2]:
!az login -o table

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"

(firefox:28809): LIBDBUSMENU-GLIB-WARNING **: Unable to get session bus: Error spawning command line 'dbus-launch --autolaunch=e05d81ec9eea4cb7abfd3fd9ec6f3655 --binary-syntax --close-stderr': Child process exited with code 1
You have logged in. Now let us find all subscriptions you have access to...
CloudName    IsDefault    Name                                            State     TenantId
-----------  -----------  ----------------------------------------------  --------  ------------------------------------
AzureCloud   False        Boston DS Dev                                   Enabled   72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Solution Template Testing                       Enabled   72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        CTS-DS                                          Disabled  72f988bf-86f1-41af-91ab-2d7cd011db47

In [3]:
!az account set --subscription $selected_subscription

In [4]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "e984a9db-1a27-4f54-98fc-282cf0dcda04",
  "isDefault": true,
  "name": "Boston-DS-Brandon-Dev",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "mabou@microsoft.com",
    "type": "user"
  }
}


You will also need to register the container service resources on your subscription if you haven't already done so.

In [5]:
!az provider register -n Microsoft.ContainerService

Registering is still on-going. You can monitor using 'az provider show -n Microsoft.ContainerService'


In [6]:
!az provider show -n Microsoft.ContainerService

{
  "authorization": {
    "applicationId": "7319c514-987d-4e9b-ac3d-d38c4f427f4c",
    "managedByRoleDefinitionId": "8e3af657-a8ff-443c-a75c-2fe8c4bcb635",
    "roleDefinitionId": "1b4a0c7f-2217-416f-acfa-cf73452fdc1c"
  },
  "id": "/subscriptions/e984a9db-1a27-4f54-98fc-282cf0dcda04/providers/Microsoft.ContainerService",
  "namespace": "Microsoft.ContainerService",
  "registrationState": "Registered",
  "resourceTypes": [
    {
      "aliases": null,
      "apiVersions": [
        "2017-07-01",
        "2017-01-31",
        "2016-09-30",
        "2016-03-30"
      ],
      "capabilities": "None",
      "locations": [
        "Japan East",
        "Central US",
        "East US 2",
        "Japan West",
        "East Asia",
        "South Central US",
        "Australia East",
        "Australia Southeast",
        "Brazil South",
        "Southeast Asia",
        "West US",
        "North Central US",
        "West Europe",
        "North Europe",
        "East US",
        "UK West"

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [7]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/e984a9db-1a27-4f54-98fc-282cf0dcda04/resourceGroups/maboumlaksd",
  "location": "eastus",
  "managedBy": null,
  "name": "maboumlaksd",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster  with 5 nodes in the resource group we created earlier.

In [ ]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 5 --generate-ssh-keys -s Standard_D4_v2

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [9]:
!sudo env "PATH=$PATH" az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [4]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "maboumlaksd" as current context in /home/mabou/.kube/config


Let's verify connection by listing the nodes.

In [5]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-20727186-0   Ready     agent     47m       v1.9.9
aks-nodepool1-20727186-1   Ready     agent     47m       v1.9.9
aks-nodepool1-20727186-2   Ready     agent     47m       v1.9.9
aks-nodepool1-20727186-3   Ready     agent     47m       v1.9.9
aks-nodepool1-20727186-4   Ready     agent     47m       v1.9.9


Let's check the pods on our cluster.

In [6]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-5fc9fc6976-25hgd             1/1       Running   1          47m
kube-system   heapster-7dfdd964f9-k2bwb               2/2       Running   0          47m
kube-system   kube-dns-v20-784dd5dbd5-85flc           3/3       Running   0          47m
kube-system   kube-dns-v20-784dd5dbd5-xcwc8           3/3       Running   0          47m
kube-system   kube-proxy-2jdjg                        1/1       Running   0          47m
kube-system   kube-proxy-d5p8n                        1/1       Running   0          47m
kube-system   kube-proxy-pl5x8                        1/1       Running   0          47m
kube-system   kube-proxy-rph7l                        1/1       Running   0          47m
kube-system   kube-proxy-xgmpg                        1/1       Running   0          47m
kube-system   kube-svc-redirect-brv9k                 1/1       Running   0          47m
kube-system   kube-sv

## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the image name and cpu requests and limits for pods. We first start with  deploying 2 pods.

In [7]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "replicas":2,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-ml"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-ml",
                      "image": image_name,

                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "cpu": 1
                           },
                           "limits":{
                               "cpu": 1.25
                           }
                       }  
                  }
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-ml"
      }
   }
}

In [8]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4, sort_keys=True)
        outfile.write('\n\n')

In [9]:
write_json_to_file(app_template, 'az-ml.json')

In [10]:
write_json_to_file(service_temp, 'az-ml.json', mode='a')

Let's check the manifest created.

In [11]:
!cat az-ml.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-ml"
    },
    "spec": {
        "replicas": 2,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-ml"
                }
            },
            "spec": {
                "containers": [
                    {
                        "image": "fboylu/mlaksdep",
                        "name": "azure-ml",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                        "resources": {
                            "limits": {
                                "cpu": 1.25
                            },
                            "requests": {
                                "cpu": 1
                            }
                        }
              

Next, we will use kubectl create command to deploy our application.

In [12]:
!kubectl create -f az-ml.json

deployment.apps/azure-ml created
service/azure-ml created


Let's check if the pod is deployed.

In [17]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-ml-7d9b465b7f-bvd9j               1/1       Running   0          7m
default       azure-ml-7d9b465b7f-nmr2s               1/1       Running   0          7m
kube-system   azureproxy-5fc9fc6976-25hgd             1/1       Running   1          55m
kube-system   heapster-7dfdd964f9-k2bwb               2/2       Running   0          55m
kube-system   kube-dns-v20-784dd5dbd5-85flc           3/3       Running   0          55m
kube-system   kube-dns-v20-784dd5dbd5-xcwc8           3/3       Running   0          55m
kube-system   kube-proxy-2jdjg                        1/1       Running   0          55m
kube-system   kube-proxy-d5p8n                        1/1       Running   0          55m
kube-system   kube-proxy-pl5x8                        1/1       Running   0          55m
kube-system   kube-proxy-rph7l                        1/1       Running   0          55m
kube-system   kube-prox

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [18]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                    SOURCE                                 MESSAGE
57m         57m          1         aks-nodepool1-20727186-0.15463987a6a9c786    Node                                     Normal    Starting                  kubelet, aks-nodepool1-20727186-0      Starting kubelet.
56m         57m          8         aks-nodepool1-20727186-0.15463987a96e94ad    Node                                     Normal    NodeHasSufficientDisk     kubelet, aks-nodepool1-20727186-0      Node aks-nodepool1-20727186-0 status is now: NodeHasSufficientDisk
56m         57m          8         aks-nodepool1-20727186-0.15463987a96ef8ae    Node                                     Normal    NodeHasSufficientMemory   kubelet, aks-nodepool1-20727186-0      Node aks-nodepool1-20727186-0 status is now: NodeHasSufficientMemory
56m         57m          7         aks-nodepool1-20727

Check the logs for the application pod.

In [19]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [20]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-07-30 19:36:54,962 CRIT Supervisor running as root (no user in config file)
2018-07-30 19:36:54,964 INFO supervisord started with pid 1
2018-07-30 19:36:55,966 INFO spawned: 'program_exit' with pid 9
2018-07-30 19:36:55,968 INFO spawned: 'nginx' with pid 10
2018-07-30 19:36:55,970 INFO spawned: 'gunicorn' with pid 11
2018-07-30 19:36:57,049 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
{"level": "INFO", "stack_info": null, "logger": "model_driver", "host": "azure-ml-7d9b465b7f-bvd9j", "timestamp": "2018-07-30T19:36:57.441858Z", "path": "/code/driver.py", "tags": [], "message": "Model object loading time: 554.34 ms"}
Initialising
{"msg": " * Running on %s://%s:%d/ %s", "level": "INFO", "stack_info": null, "logger": "werkzeug", "host": "azure-ml-7d9b465b7f-bvd9j", "timestamp": "2018-07-30T19:36:57.448699Z", "path": "/opt/conda/envs/py3.5/lib/python3.5/site-packages/werkzeug/_internal.py", "tags": [], "message": " * Running o

In [21]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-ml   2         2         2            2           7m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [22]:
!kubectl get service azure-ml

NAME       TYPE           CLUSTER-IP    EXTERNAL-IP   PORT(S)        AGE
azure-ml   LoadBalancer   10.0.167.16   40.87.43.89   80:30234/TCP   7m


# Scaling

In this part, we provide commands to scale the number of pods.

In [52]:
!kubectl scale --current-replicas=2 --replicas=35 deployment/azure-ml

deployment.extensions/azure-ml scaled


In [55]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-ml-7d9b465b7f-245fv               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-274qk               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-4h5pd               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-5qwlz               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-66lcp               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-72dww               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-7cx6l               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-8hzwc               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-dwg6p               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-dwlhg               1/1       Running   0          1h
default       azure-ml-7d9b465b

In [56]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-ml   35        35        35           35          1h


Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb). Once, we are done with all the notebooks of the tutorial, below instructions can be used to delete the cluster and free resources

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [48]:
!kubectl delete -f az-ml.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [3]:
!az aks delete -n $aks_name -g $resource_group -y

In [4]:
!az group delete --name $resource_group -y

 - Finished ..